실행 위치: 로컬

In [ ]:
# PYTHONPATH 설정
import sys, os 

BASE_DIR = os.getcwd()
for _ in range(1):
    BASE_DIR = os.path.dirname(BASE_DIR)

sys.path.insert(0, BASE_DIR)
print(sys.path)


# 1. Tool 함수 정의

In [ ]:
from tools.realtime_weather_api_tool.weather_api_tool import WeatherApiTool

weather_api_tool = WeatherApiTool()

In [ ]:
tool_functions = {"get_weather_api": weather_api_tool.get_weather_api}
tool_description = [weather_api_tool.tools_description]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "지금 제주시 날씨는 어떄? 적합한 야외 활동 추천해줘"
    }
]


# 2. 추론 수행 (툴 제공)

In [ ]:
from fastapi_app.app.core.config import settings

vllm_ip_port = settings.VLLM_IP_PORT.rstrip("/")
vllm_api_key = settings.VLLM_API_KEY
vllm_chat_completions_url = f"{vllm_ip_port}/v1/chat/completions"

payload = {
    "model": "openai/gpt-oss-20b",
    "messages": messages,
    "tools": tool_description,
    "tool_choice": "auto",  # 모델이 스스로 판단하여 툴 사용 여부 결정
    "temperature": 0,
    "stream": True
}

headers={"Content-Type": "application/json", "Authorization": f"Bearer {vllm_api_key}"}

# 3. 응답 확인

In [ ]:
import httpx

async with httpx.AsyncClient(timeout=30) as client:
    async with client.stream("POST", url=vllm_chat_completions_url, json=payload, headers=headers) as response:
        response.raise_for_status()
        async for line in response.aiter_lines():
            if line:
                print(line)
        

- 모델의 streaming 응답과 정적 응답의 포맷 자체는 다르지 않으나
- Streaming 응답인 경우 tool_calls 부분이 chunk로 나뉘어 옴에 유의
- 또한 맨 마지막 chunk는 finish_reason = "tool_calls" 로 작성됨에 주목